### Extração Manual

In [116]:
%%capture
%pip install pandas matplotlib requests fastparquet

In [117]:
import zipfile, requests, os
import pandas as pd
import matplotlib.pyplot as plt
import fastparquet

pd.options.display.float_format = '{:.2f}'.format

In [118]:
FLOAT_COLS = [
    "a_vencer_ate_90_dias",
    "a_vencer_de_91_ate_360_dias",
    "a_vencer_de_361_ate_1080_dias",
    "a_vencer_de_1081_ate_1800_dias",
    "a_vencer_de_1801_ate_5400_dias",
    "a_vencer_acima_de_5400_dias",
    "vencido_acima_de_15_dias",
    "carteira_ativa",
    "carteira_inadimplida_arrastada",
    "ativo_problematico"
]

CATEGORY_COLS = [
    'uf',
    'tcb',
    'sr',
    'cliente'
    'ocupacao',
    'cnae_secao',
    'cnae_subclasse',
    'cliente', 
    'ocupacao',
    'porte',
    'modalidade',
    'origem',
    'indexador'
]

INT_COLS = [
    'numero_de_operacoes'
]

def convert_category_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].astype('category')

    return df

def convert_float_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].apply(lambda x : str(x).replace(",","."))
    df[column] = df[column].astype('float64')

    return df

def convert_int_dtype(df: pd.DataFrame, column: str):

    df[column] = df[column].apply(lambda x : str(x).replace("<= 15","15"))
    df[column] = df[column].astype(int)

    return df

In [119]:
planilhas = [
	'planilha_2012',
	'planilha_2013',
	'planilha_2014',
	'planilha_2015',
	'planilha_2016',
	'planilha_2017',
	'planilha_2018',
	'planilha_2019',
	'planilha_2020',
	'planilha_2021',
	'planilha_2022',
	'planilha_2023',
	'planilha_2024'
]

for planilha in planilhas:
	if not os.path.exists(f'data/{planilha}.zip'):
		url = f'https://www.bcb.gov.br/pda/desig/{planilha}.zip'

		response = requests.get(url)

		with open(f'data/{planilha}.zip', 'wb') as f:
			f.write(response.content)

In [120]:
if not os.path.exists('data/planilhas'):
	for planilha in planilhas:
		zip_obj = zipfile.ZipFile(f'data/{planilha}.zip', 'r')
		zip_obj.extractall(f'data/planilhas')
		print(f'Extraído {planilha}')

In [121]:
files = os.listdir('data/planilhas')
files.sort()
df = pd.DataFrame()
for file in files:
    temp_df = pd.read_csv(f'data/planilhas/{file}', sep=';', encoding='utf-8', nrows=15000)
    df = pd.concat([df, temp_df])
    print(f'Processado o arquivo {file}, com {len(temp_df)} linhas e {len(temp_df.columns)} colunas')

Processado o arquivo planilha_201206.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201207.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201208.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201209.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201210.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201211.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201212.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201301.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201302.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201303.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201304.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201305.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201306.csv, com 15000 linhas e 23 colunas
Processado o arquivo planilha_201307.csv, com 15000 linhas e 23 

In [122]:
if not os.path.exists('data/raw'):
	os.mkdir('data/raw')
df.to_parquet('data/raw/df.parquet')

## AJUSTE TIPAGEM DOS DADOS

In [123]:
df['uf']
df['carteira_inadimplida_arrastada']

0           616,49
1         91240,63
2          1959,96
3        411759,93
4        115456,50
           ...    
14995      5415,94
14996         0,00
14997       486,95
14998     20049,84
14999         0,00
Name: carteira_inadimplida_arrastada, Length: 2190000, dtype: object

In [124]:
df['data_base'] = pd.to_datetime(df['data_base'], format="%Y-%m-%d")

for column in df.columns:

	if column in FLOAT_COLS:
		df = convert_float_dtype(df=df, column=column)
		df = df.rename(columns={column: "vl_" + column.lower()})

	if column in CATEGORY_COLS:
		df = convert_category_dtype(df=df, column=column)
		df = df.rename(columns={column: "ct_" + column.lower()})

	if column in INT_COLS:
		df = convert_int_dtype(df=df, column=column)
		df = df.rename(columns={column: "nu_" + column.lower()})

In [125]:
df.columns

Index(['data_base', 'ct_uf', 'ct_tcb', 'ct_sr', 'ct_cliente', 'ct_ocupacao',
       'ct_cnae_secao', 'ct_cnae_subclasse', 'ct_porte', 'ct_modalidade',
       'ct_origem', 'ct_indexador', 'nu_numero_de_operacoes',
       'vl_a_vencer_ate_90_dias', 'vl_a_vencer_de_91_ate_360_dias',
       'vl_a_vencer_de_361_ate_1080_dias', 'vl_a_vencer_de_1081_ate_1800_dias',
       'vl_a_vencer_de_1801_ate_5400_dias', 'vl_a_vencer_acima_de_5400_dias',
       'vl_vencido_acima_de_15_dias', 'vl_carteira_ativa',
       'vl_carteira_inadimplida_arrastada', 'vl_ativo_problematico',
       '﻿data_base'],
      dtype='object')

In [126]:
df['ct_porte'].value_counts()

ct_porte
PJ - Micro                                       669342
PJ - Pequeno                                     563754
PJ - Médio                                       288752
PF - Mais de 5 a 10 salários mínimos              88965
PF - Mais de 3 a 5 salários mínimos               85810
PF - Mais de 1 a 2 salários mínimos               83242
PF - Mais de 2 a 3 salários mínimos               81845
PF - Mais de 10 a 20 salários mínimos             80033
PF - Acima de 20 salários mínimos                 75983
PF - Até 1 salário mínimo                         69558
PF - Sem rendimento                               38010
PJ - Grande                                       27588
PF - Indisponível                                 23280
PJ - Indisponível                                 13838
Name: count, dtype: int64

Vamos separar coluna ct_porte em duas colunas:

ct_classificacao , que é o substring(0:2) de ct_porte

ct_tamanho , que é o substring(5:) de ct_porte

In [127]:
df['ct_classificacao'] = df['ct_porte'].apply(lambda x: x[0:2])
df['ct_porte'] = df['ct_porte'].apply(lambda x: x[5:])

In [128]:
df['ct_modalidade'].value_counts()

ct_modalidade
PJ - Capital de giro                                                              391649
PJ - Outros créditos                                                              339850
PJ - Cheque especial e conta garantida                                            274245
PJ - Investimento                                                                 220932
PJ - Financiamento de infraestrutura/desenvolvimento/projeto e outros créditos    208129
PF - Outros créditos                                                              152418
PJ - Operações com recebíveis                                                     109568
PF - Cartão de crédito                                                            106873
PF - Empréstimo sem consignação em folha                                          101216
PF - Empréstimo com consignação em folha                                           78727
PF - Veículos                                                                      72736
PF - Ru

In [129]:
df['ct_modalidade'] = df['ct_modalidade'].apply(lambda x: x[5:])

In [130]:
df['ct_ocupacao'].value_counts()

ct_ocupacao
-                                                  1563274
PF - Outros                                         116780
PF - Servidor ou empregado público                   98260
PF - Empresário                                      92430
PF - Autônomo                                        87336
PF - Empregado de empresa privada                    85104
PF - Aposentado/pensionista                          82621
PF - MEI                                             37027
PF - Empregado de entidades sem fins lucrativos      27168
Name: count, dtype: int64

In [131]:
df['ct_ocupacao'] = df['ct_ocupacao'].apply(lambda x: x[5:])

In [132]:
df['vl_carteira_ativa'].describe()

count      2190000.00
mean       2439018.73
std       24629714.65
min              0.01
25%          12727.54
50%          66187.07
75%         330177.21
max     2088395883.83
Name: vl_carteira_ativa, dtype: float64

In [133]:
df['vl_carteira_ativa_n_arrastada'] = df['vl_carteira_ativa'] - df['vl_carteira_inadimplida_arrastada']

In [134]:
df['vl_carteira_ativa_n_arrastada'].describe()

count      2190000.00
mean       2346042.91
std       24145767.41
min              0.00
25%           9533.15
50%          56521.72
75%         300619.19
max     2088395883.83
Name: vl_carteira_ativa_n_arrastada, dtype: float64

In [135]:
df.dtypes

data_base                            datetime64[ns]
ct_uf                                      category
ct_tcb                                     category
ct_sr                                      category
ct_cliente                                 category
ct_ocupacao                                category
ct_cnae_secao                              category
ct_cnae_subclasse                          category
ct_porte                                     object
ct_modalidade                                object
ct_origem                                  category
ct_indexador                               category
nu_numero_de_operacoes                        int64
vl_a_vencer_ate_90_dias                     float64
vl_a_vencer_de_91_ate_360_dias              float64
vl_a_vencer_de_361_ate_1080_dias            float64
vl_a_vencer_de_1081_ate_1800_dias           float64
vl_a_vencer_de_1801_ate_5400_dias           float64
vl_a_vencer_acima_de_5400_dias              float64
vl_vencido_a

In [136]:
os.makedirs('data/trusted', exist_ok=True)
df.to_parquet('data/trusted/df.parquet')